In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountains

In [2]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [3]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [4]:
print(fountains.crs)

None


In [5]:
fountains = fountains.set_crs('epsg:4326')

In [6]:
print(fountains.crs)

epsg:4326


In [7]:
columns = ["precision_localisation", "date_installation", "etat", "x", "y"]
fountains = fountains.drop(columns=columns)

In [8]:
fountains.head(1)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (-73.58946 45.59201)


### Ruelles vertes

In [12]:
# rvertes = gpd.read_file(os.getenv("ruelles_vertes"))
rvertes = gpd.read_file("ruelles_vertes.geojson")
rvertes.columns = rvertes.columns.str.lower()

In [14]:
print(rvertes.geom_type[0])

Polygon


In [15]:
print(rvertes.crs)

EPSG:4326


In [16]:
rvertes.head(1)

,id_trc,proprietaire_ref,code_arr,forme,ruelle_id,date_amenagement,date_bonification,circulation,type_amenagement,type_recouvrement,superficie_ruelle_m2,superficie_verdissement_m2,numero_de_resolution,date_maj,geometry
0,403365,Plateau - Mont-Royal,PMR,A,PMR-A-1000079,20170901,NaN,Ouverte,Plates-bandes et mobilier,Asphalté ou bétonnée,218.545,4.7,NaN,20191113,"POLYGON ((-73.58180 45.52976, -73.58185 45.529..."


In [17]:
columns = ["id_trc", "proprietaire_ref", "type_amenagement", "geometry"]
rvertes = rvertes.filter(columns)

### calcul the colsest distance between fountain and ruelles vertes

In [18]:
fountains_utm = fountains.to_crs(epsg=32188)
rvertes_utm = rvertes.to_crs(epsg=32188)

In [19]:
print(fountains_utm.crs)
print(rvertes_utm.crs)

EPSG:32188
EPSG:32188


In [20]:
fountain_w_rvertes_data = fountains_utm.sjoin_nearest(rvertes_utm, distance_col="nearest_rvertes")

In [21]:
print(f"min: {min(fountain_w_rvertes_data['nearest_rvertes'])}")
print(f"max: {max(fountain_w_rvertes_data['nearest_rvertes'])}")

min: 2.5745363322386083
max: 23151.850528372557


In [22]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [24]:
part = fountain_w_rvertes_data[fountain_w_rvertes_data.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [25]:
m

In [26]:
folium.GeoJson(
    rvertes
).add_to(m)

In [27]:
m

In [28]:
fountain_w_rvertes_data[fountain_w_rvertes_data.nom_parc_lieu.str.contains("Royal", na=False)]

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry,index_right,id_trc,proprietaire_ref,type_amenagement,nearest_rvertes
830,844,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.591708,45.502730,POINT (297633.330 5040316.150),869,403375,Plateau - Mont-Royal,Plates-bandes et mobilier,1533.656800
832,846,Ville-Marie,du Mont-Royal,chalet,NaN,Grands Parcs,-73.588005,45.504188,POINT (297922.910 5040477.850),869,403375,Plateau - Mont-Royal,Plates-bandes et mobilier,1404.961763
834,848,Ville-Marie,du Mont-Royal,chemin,NaN,Grands Parcs,-73.589275,45.504000,POINT (297823.630 5040457.050),869,403375,Plateau - Mont-Royal,Plates-bandes et mobilier,1407.520003
836,850,Ville-Marie,du Mont-Royal,stationnement,NaN,Grands Parcs,-73.593449,45.501453,POINT (297497.160 5040174.420),869,403375,Plateau - Mont-Royal,Plates-bandes et mobilier,1679.818374
842,856,Ville-Marie,du Mont-Royal,NaN,ave du Parc/ave Duluth Ouest,Grands Parcs,-73.585721,45.514257,POINT (298102.570 5041596.630),562,403528,Plateau - Mont-Royal,Plates-bandes,416.067831
843,857,Ville-Marie,du Mont-Royal,NaN,ave du Parc/ave Duluth Ouest,Grands Parcs,-73.585019,45.513926,POINT (298157.360 5041559.740),562,403528,Plateau - Mont-Royal,Plates-bandes,436.680630
620,631,Ville-Marie,près du parc du Mont-Royal,NaN,ave des Pins Ouest/rue Peel,au pied de l'escalier,-73.582003,45.504592,POINT (298391.930 5040522.260),611,407044,Plateau - Mont-Royal,Plates-bandes,1171.613536
841,855,Ville-Marie,du Mont-Royal,NaN,ave des Pins Ouest/rue Peel,Grands Parcs,-73.583063,45.505103,POINT (298309.170 5040579.060),611,407044,Plateau - Mont-Royal,Plates-bandes,1149.685891
833,847,Ville-Marie,du Mont-Royal,chalet,près du lac des castors,Grands Parcs,-73.599150,45.498618,POINT (297051.250 5039859.870),805,404855,Ville-Marie,Plates-bandes,1759.855345
835,849,Ville-Marie,du Mont-Royal,sentier,près du lac des castors,Grands Parcs,-73.599359,45.498969,POINT (297034.910 5039898.870),805,404855,Ville-Marie,Plates-bandes,1798.434809
